# **Handwritten Digit Classification**
Submitted By:  
> **Group 19**  
*18EE30021 : Pratyush Jaiswal*    
*18EE30029 : Nuruddin Jiruwala*       
*18EE10043 : Rhitvik Sinha*  
*18EE10062 : Akshat Kumar Gupta*  




In [ ]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
import matplotlib.pyplot as plt
import random
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


## Flattening dataset into 784-dimensional binary vectors and Keeping aside 20% data for testing and another 20% for validation

In [ ]:
X_train=np.concatenate((X_train,X_test))

In [ ]:
X_train.shape

(70000, 28, 28)

In [ ]:
Y_train=np.concatenate((y_train,y_test))

In [ ]:
Y_train.shape

(70000,)

In [ ]:
Train=[]
threshold = 127
for i in range(X_train.shape[0]):
  temp=X_train[i].flatten()                   # flattening and binarization of data
  temp=np.where(temp>threshold, 1, 0)
  Train.append(temp)
Train=np.array(Train)

In [ ]:
Train.shape

(70000, 784)

In [ ]:
trainx=Train[:int(0.6*Train.shape[0])]
trainy=Y_train[:int(0.6*Y_train.shape[0])]
validationx=Train[int(0.6*Train.shape[0]):int(0.8*Train.shape[0])]
validy=Y_train[int(0.6*Y_train.shape[0]):int(0.8*Y_train.shape[0])]
testx=Train[int(0.8*Train.shape[0]):int(1*Train.shape[0])]
testy=Y_train[int(0.8*Y_train.shape[0]):int(1*Y_train.shape[0])]

## Building a decision tree (using library function) based on 10 dimensions only

In [ ]:
indices=random.sample(range(0, 784), 10)

In [ ]:
tree1x=trainx[:,indices]
validationtreex=validationx[:,indices]

In [ ]:
tree1x.shape

(42000, 10)

In [ ]:
trainy.shape

(42000,)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
treem=DecisionTreeClassifier(criterion='entropy',max_depth=5)
treem.fit(tree1x,trainy)  # the input shapes must be (42000,10) and (42000,)  i.e. (num_labels,num_features) for X and (num_labels) for Y

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
predict1=treem.predict(validationtreex)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy=accuracy_score(predict1,validy)

In [ ]:
accuracy

0.33714285714285713

## Building a decision tree of max. depth 5 for each of the 50 random subsets like above, each of dimension 10

In [ ]:
def createtree(trainx,trainy,validationx,validy):
  indices=random.sample(range(trainx.shape[1]), 10) # random 10 features
  tree1x=trainx[:,indices]
  validationtreex=validationx[:,indices]
  model=DecisionTreeClassifier(max_depth=5)
  model.fit(tree1x,trainy) # predicting on those features
  ypred=model.predict(validationtreex)
  accuracy=accuracy_score(ypred,validy)
  return accuracy,indices,model

In [ ]:
tree_acc={} # accuracies(weights) for each tree
trees={}    # model themselve
tree_feat={} # features selected for the tree
for i in range(50):
  tree_acc[i],tree_feat[i],trees[i]=createtree(trainx,trainy,validationx,validy) # getting all 50 trees dataset

In [ ]:
tree_acc

{0: 0.3359285714285714,
 1: 0.38221428571428573,
 2: 0.29035714285714287,
 3: 0.373,
 4: 0.31685714285714284,
 5: 0.24357142857142858,
 6: 0.41864285714285715,
 7: 0.27085714285714285,
 8: 0.3142142857142857,
 9: 0.3950714285714286,
 10: 0.195,
 11: 0.33635714285714285,
 12: 0.24521428571428572,
 13: 0.3005,
 14: 0.33864285714285713,
 15: 0.33135714285714285,
 16: 0.22378571428571428,
 17: 0.2832142857142857,
 18: 0.3585,
 19: 0.237,
 20: 0.2567857142857143,
 21: 0.3597142857142857,
 22: 0.2475,
 23: 0.36407142857142855,
 24: 0.3565,
 25: 0.38285714285714284,
 26: 0.30342857142857144,
 27: 0.32435714285714284,
 28: 0.2922142857142857,
 29: 0.36457142857142855,
 30: 0.25685714285714284,
 31: 0.27064285714285713,
 32: 0.2392857142857143,
 33: 0.348,
 34: 0.2724285714285714,
 35: 0.32807142857142857,
 36: 0.3417142857142857,
 37: 0.33,
 38: 0.28714285714285714,
 39: 0.3454285714285714,
 40: 0.3274285714285714,
 41: 0.25435714285714284,
 42: 0.37764285714285717,
 43: 0.42457142857142854,
 

## Carrying out weighted classification of the test set using these 50 decision trees, along with their validation accuracies as weights

In [ ]:
count=0
for i in range(testx.shape[0]):
  probs=np.zeros((1,10))
  for key in tree_acc.keys():
    test=testx[i][tree_feat[key]].reshape(1,-1) # getting the feature numbers for the tree required
    probs+=tree_acc[key]*trees[key].predict_proba(test) # summing probabilities of all labels
  if testy[i]==np.argmax(probs): # finding max probability(can be greater than 1 too) of 10 labels
    count+=1
accuracy=count/testx.shape[0]


In [ ]:
accuracy

0.8069285714285714

## Implementing Ensemble Adaboost Algorithm from Scratch

In [ ]:
def update_weights(weights,error):
  classes=10
  e=np.sum(error)
  #print(e)
  alpha=0.5*(np.log((1-e)/e) + np.log(classes-1)) # calculate multiclass alpha
  #print(alpha)
  for i in range(weights.shape[0]):
    if error[i]!=0:
      weights[i]*=np.exp(alpha)       # if wrong prediction increase weight
    else:
      weights[i]*=np.exp(-alpha)
  weights=weights/np.sum(weights) # normalize
  return alpha,weights

In [ ]:
## initialize weights
N=trainx.shape[0]
weights=np.ones(N)/N;
# train on our random forest and get training error
count=0
init_error=np.zeros(N); # binary vector of misclassified outputs
for i in range(trainx.shape[0]):
  probs=np.zeros((1,10))
  for key in tree_acc.keys():
    train=trainx[i][tree_feat[key]].reshape(1,-1) # getting the feature numbers for the tree required
    probs+=tree_acc[key]*trees[key].predict_proba(train) # summing probabilities of all labels
  if trainy[i]==np.argmax(probs): # finding max probability(can be greater than 1 too) of 10 labels
    count+=1
  else:
    init_error[i]=1


In [ ]:
w_cpy=weights
err_cpy=init_error
initial_error=np.multiply(w_cpy,err_cpy) # weighted error
base_perf,weights=update_weights(w_cpy,initial_error) # base classifier weight and alpha

In [ ]:
perf={}
ada={}
for i in range(20):
  error=np.zeros(N)
  treea=DecisionTreeClassifier(criterion='entropy',max_depth=5)
  treea.fit(trainx,trainy,sample_weight=weights)
  ypred=treea.predict(trainx)
  ada[i]=treea
  for j in range(N):
    if trainy[j]!=ypred[j]:
      error[j]=1
  finerror=np.multiply(weights,error) # update error
  perf[i],weights=update_weights(weights,finerror) # update alphas and weights

## Using this ensemble, carrying out classification on the test set and reporting accuracy

In [ ]:
def random_predict(X):
  probs=np.zeros((1,10))
  for key in tree_acc.keys():
    temp=X[tree_feat[key]].reshape(1,-1) # getting the feature numbers for the tree required
    probs+=tree_acc[key]*trees[key].predict_proba(temp) # summing probabilities of all labels
  return np.argmax(probs) # return prediction for given X datapoint

In [ ]:
ypred=np.zeros(testx.shape[0])
for i in range(testx.shape[0]):
  prob=np.zeros(10)
  prob[random_predict(testx[i])]+=base_perf # adding the base alpha for the randome forest predicted output
  for key in ada.keys():
    prob[ada[key].predict(testx[i].reshape(1,-1))]+=perf[key]  # summing alphas for each label
  ypred[i]=np.argmax(prob)

In [ ]:
accuracy=accuracy_score(ypred,testy)

In [ ]:
accuracy

0.8609285714285714